In [ ]:
!pip install ipykernel
!pip install --upgrade nbformat
!pip install bokeh
!pip install plotly

Biểu đồ dân số từng nước theo từng năm 

In [290]:
from bokeh.plotting.figure import figure
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_csv('./population.csv')


In [291]:
fig = px.bar(df, x="Country", y="Population", animation_frame="Year", animation_group="Country", color="Continent", log_y=True,range_y=[10,10000000000],height=900)


fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
fig.update_xaxes(tickangle=-45)
sliders=fig['layout']['sliders']
updatemenus=fig['layout']['updatemenus']
fig['layout']['updatemenus'][0]['pad']=dict(r= 20, t= 270)
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 250)
fig.update_layout(
    yaxis={"tickmode": "array",
          "tickvals": pd.to_numeric([f"{n:.1g}" for n in np.geomspace(1, df["Population"].max(), 25)])
           })
fig.update_layout(
    title={
        'text': "Biểu đồ thống kê dân số các nước trên thế giới",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

In [292]:
continent_df=df.sort_values(by=['Continent'],ascending=True)[['Year','Population','Continent']].groupby(['Year','Continent'],group_keys=True).sum().reset_index()
fig = px.area(continent_df
, x="Year", y="Population", color="Continent", line_group="Continent")
for i in range(len(fig['data'])):
  fig['data'][i]['line']['width']=0
fig.update_layout(
    title={
        'text': "Biểu đồ phân miền dân số thế giới theo châu lục",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [293]:

fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

world_pop={i : sum(df[df['Year']==i]['Population']) for i in df['Year'].unique()}

df_=df[df['Year']==1955]
df_.index=df_.pop('Country')
df_=df_[['Population']].T

continent_df_=continent_df[continent_df['Year']==1955]
continent_df_.index=continent_df_.pop('Continent')
continent_df_=continent_df_[['Population']].T


fig_dict["data"] = [go.Pie(labels=df_.columns,
                           title="Population by country : 1955",
                           titleposition='bottom center',
                           values=df_.iloc[0],
                           sort=True,
                           domain={'x': [0.5, 1], 'y': [0, 0.5+0.5*(world_pop[1955]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Country=%{label}<br>Population=%{value}<extra></extra>'),

                        go.Pie(labels=continent_df_.columns,
                           title="Population by continent : 1955",
                           titleposition='bottom center',
                           values=continent_df_.iloc[0],
                           sort=True,
                           domain={'x': [0, 0.5], 'y': [0.0, 0.5+0.5*(world_pop[1955]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Continent=%{label}<br>Population=%{value}<extra></extra>')]
frames_list = []
sliders_dict ={'steps':[]}
for index in df['Year'].unique() :
    df_=df[df['Year']==index]
    df_.index=df_.pop('Country')
    df_=df_[['Population']].T
    continent_df_=continent_df[continent_df['Year']==index]
    continent_df_.index=continent_df_.pop('Continent')
    continent_df_=continent_df_[['Population']].T
    frames_list.append(
        go.Frame(data=[go.Pie(labels=df_.columns,
                           title="Population by country : "+str(index),
                           titleposition='bottom center',
                           values=df_.iloc[0],
                           sort=True,
                           domain={'x': [0.5, 1], 'y': [0.0, 0.5+0.5*(world_pop[index]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Country=%{label}<br>Population=%{value}<extra></extra>'),
                        go.Pie(labels=continent_df_.columns,
                           title="Population by continent : "+str(index),
                           titleposition='bottom center',
                           values=continent_df_.iloc[0],
                           sort=True,
                           domain={'x': [0.0, 0.5], 'y': [0.0, 0.5+0.5*(world_pop[index]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Continent=%{label}<br>Population=%{value}<extra></extra>')]
                           ,
                name= str(index))
    )
    slider_step = {'args': [
        [index],

       {'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate', 'fromcurrent': True, 'transition':
                        {'duration': 0, 'easing': 'linear'}}
     ],
     'label': str(index),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

# create buttons for animations : Play, Pause
fig_dict["layout"]["sliders"] =sliders
fig_dict["layout"]["updatemenus"] =updatemenus
fig_dict['layout']['updatemenus'][0]['pad']=dict(r= 20, t= 0)
fig_dict['layout']['sliders'][0]['pad']=dict(r= 10, t= 0)
# add our list of frames to our dictionary
fig_dict["frames"] = frames_list
# create final figure
fig = go.Figure(fig_dict)
fig.update_layout(height=int(600))
fig.update_layout(
    title={
        'text': "Biểu đồ phần trăm thị phần dân số thế giới theo lục địa và đất nước",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()




In [322]:
fig=px.scatter(df, x="Population", y="UrbanPop%", animation_frame="Year", animation_group="Country",
        color="Continent", hover_name="Country",
           log_x=True, range_x=[100,10000000000], range_y=[2,105])

fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện mối liên hệ giữa dân số và tỉ lệ dân số ở thành thị",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [323]:
fig=px.scatter(df, x="Population", y="MedianAge", animation_frame="Year", animation_group="Country",
        color="Continent", hover_name="Country",
           log_x=True, range_x=[10000,10000000000], range_y=[10,50])
fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện mối liên hệ giữa dân số và độ tuổi trung bình",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()